In [1]:
import pandas as pd
import numpy as np

In [2]:
companies_selected = pd.read_excel("Companies.xlsx")

In [19]:
companies_selected.shape

(79, 13)

In [3]:
companies_selected.head()

,Company Name,Industry,Beta Value,Grade,TSR
0,NextEra Energy (NEE),Utilities,0.49,B,53.47
1,Duke Energy (DUK),Utilities,0.38,C,57.51
2,Atmos Energy (ATO),Utilities,0.50,B,54.13
3,Avangrid (AGR),Utilities,0.34,A,-8.57
4,Otter Tail (OTTR),Utilities,0.69,A,87.72


In [4]:
# since TSR is in % we will grade wight in % as well
grade_weight = {"A": 90, "B": 80, "C": 70, "D": 60, "E": 50, "F": 40}
companies_selected["GradeWeight"] = companies_selected["Grade"].map(grade_weight)

In [15]:
def calculate_grade_score(data, x):
    return (1 - x) * data["Weighted TSR"] + x * data["GradeWeight"]

In [16]:
def monte_carlo_simulation(
    data, max_industry_allocation=0.16, max_stock_allocation=0.045, target_beta=1.0
):
    best_portfolio = None
    best_portfolio_beta = float("inf")
    best_run = None

    for iteration in range(
        65000, 80000
    ):  # We want to keep the grade wight at a minumum of 0.65 of the total caluclation
        weight_factor = iteration / 100000  # Fraction for Grade Score calculation

        # Normalize TSR:
        data["Industry TSR total"] = data.groupby("Industry")["TSR"].transform("sum")
        data["Weighted TSR"] = data["TSR"] / data["Industry TSR total"]

        # Calculate Grade Score:
        data["Grade Score"] = data.apply(
            calculate_grade_score, axis=1, args=(weight_factor,)
        )

        # Calculate Weight:
        data["Weight"] = data["Grade Score"] / data["Grade Score"].sum()

        # Apply industry-level restrictions:
        data["Industry Allocation"] = data.groupby("Industry")["Weight"].transform(
            "sum"
        )
        data["Allocation_by_restriction"] = np.where(
            data["Industry Allocation"] > max_industry_allocation,
            max_industry_allocation,
            data["Industry Allocation"],
        )

        # Normalize Weights based on restrictions:
        data["Normalised Weight"] = np.where(
            data["Allocation_by_restriction"] == max_industry_allocation,
            (data["Weight"] / data["Industry Allocation"]) * max_industry_allocation,
            data["Weight"],
        )

        # Calculate Portfolio Beta:
        portfolio_beta = (data["Beta Value"] * data["Normalised Weight"]).sum()

        # Check if this portfolio is better:
        if portfolio_beta < best_portfolio_beta:
            test = weight_factor
            best_portfolio_beta = portfolio_beta
            best_run = iteration
            best_portfolio = data.copy()
            print(best_run, best_portfolio_beta)

    return test, best_run, best_portfolio_beta, best_portfolio

In [17]:
test, run_no, final_beta, ideal_portfolio = monte_carlo_simulation(companies_selected)
print(f"best run: {run_no} final_beta: {final_beta} ")
print(ideal_portfolio)

65000 0.9228274960593076
65001 0.92282749564375
65002 0.9228274952282056
65003 0.9228274948126736
65004 0.9228274943971547
65005 0.9228274939816483
65006 0.9228274935661549
65007 0.9228274931506741
65008 0.9228274927352059
65009 0.9228274923197509
65010 0.9228274919043082
65011 0.9228274914888783
65012 0.9228274910734612
65013 0.9228274906580571
65014 0.9228274902426657
65015 0.9228274898272871
65016 0.9228274894119209
65017 0.9228274889965676
65018 0.9228274885812274
65019 0.9228274881658995
65020 0.9228274877505847
65021 0.9228274873352824
65022 0.922827486919993
65023 0.9228274865047161
65024 0.9228274860894522
65025 0.9228274856742007
65026 0.9228274852589624
65027 0.9228274848437368
65028 0.9228274844285238
65029 0.9228274840133233
65030 0.922827483598136
65031 0.9228274831829612
65032 0.922827482767799
65033 0.9228274823526498
65034 0.9228274819375133
65035 0.9228274815223892
65036 0.9228274811072789
65037 0.9228274806921802
65038 0.9228274802770944
65039 0.9228274798620214
65040

In [8]:
test

0.65

In [9]:
run_no

65000

In [10]:
final_beta

0.8796795000604941

In [11]:
ideal_portfolio

,Company Name,Industry,Beta Value,Grade,TSR,GradeWeight,Industry TSR total,Weighted TSR,Grade Score,Weight,Industry Allocation,Allocation_by_restriction,Normalised Weight
0,NextEra Energy (NEE),Utilities,0.49,B,53.47,80,358.64,0.149091,70.7145,0.009452,0.087150,0.087150,0.009452
1,Duke Energy (DUK),Utilities,0.38,C,57.51,70,358.64,0.160356,65.6285,0.008772,0.087150,0.087150,0.008772
2,Atmos Energy (ATO),Utilities,0.50,B,54.13,80,358.64,0.150931,70.9455,0.009483,0.087150,0.087150,0.009483
3,Avangrid (AGR),Utilities,0.34,A,-8.57,90,358.64,-0.023896,55.5005,0.007418,0.087150,0.087150,0.007418
4,Otter Tail (OTTR),Utilities,0.69,A,87.72,90,358.64,0.244591,89.2020,0.011923,0.087150,0.087150,0.011923
...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,COSTCO WHOLESALE CORPORATION (XNAS:COST),Consumer Staples,0.79,D,17.43,60,177.69,0.098092,45.1005,0.006028,0.077816,0.077816,0.006028
75,DOLLAR GENERAL CORPORATION (XNYS:DG),Consumer Staples,0.45,B,14.09,80,177.69,0.079295,56.9315,0.007609,0.077816,0.077816,0.007609
76,COLGATE-PALMOLIVE COMPANY (XNYS:CL),Consumer Staples,0.41,A,15.40,90,177.69,0.086668,63.8900,0.008540,0.077816,0.077816,0.008540
77,THE HERSHEY COMPANY (XNYS:HSY),Consumer Staples,0.37,B,12.95,80,177.69,0.072880,56.5325,0.007556,0.077816,0.077816,0.007556


In [14]:
ideal_portfolio.sort_values(by="Normalised Weight", ascending=False)

,Company Name,Industry,Beta Value,Grade,TSR,GradeWeight,Industry TSR total,Weighted TSR,Grade Score,Weight,Industry Allocation,Allocation_by_restriction,Normalised Weight
44,NVDA,Technology,1.76,C,2694.86,70,4829.52,0.557997,988.7010,0.132150,0.288483,0.160000,0.073294
25,Eli Lilly and Company (LLY),Healthcare,0.42,C,675.16,70,1435.04,0.470482,281.8060,0.037666,0.136636,0.136636,0.037666
40,TSLA,Technology,2.12,A,1236.11,90,4829.52,0.255949,491.1385,0.065646,0.288483,0.160000,0.036409
10,Interactive Brokers (IBKR),Finance,0.79,D,303.51,60,1525.19,0.198998,145.2285,0.019411,0.139116,0.139116,0.019411
11,LPL Financial (LPLA),Finance,0.85,C,276.95,70,1525.19,0.181584,142.4325,0.019038,0.139116,0.139116,0.019038
...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,"Hitachi, Ltd. (OTCM:HTHIY)",Industruals,1.15,D,8.50,60,125.13,0.067929,41.9750,0.005610,0.069275,0.069275,0.005610
61,"UNITED PARCEL SERVICE, INC. (XNYS:UPS)",Industruals,0.98,D,6.04,60,125.13,0.048270,41.1140,0.005495,0.069275,0.069275,0.005495
64,Siemens AG (XBUE:SIEGY),Industruals,1.22,E,8.50,50,125.13,0.067929,35.4750,0.004742,0.069275,0.069275,0.004742
48,COIN,Technology,3.34,B,-10.57,80,4829.52,-0.002189,48.3005,0.006456,0.288483,0.160000,0.003581


In [33]:
final_port = ideal_portfolio[["Company Name"]]

In [ ]:
# ---------------------------------------------------------------------- EVERYTHING BELOW IS FOR TESTING OUT PORTFOLIO (WIP CURRENTLY)

In [39]:
tickers = ["AAPL", "MSFT", "GOOG"]  # Example tickers
final_port = pd.DataFrame(index=tickers)

In [40]:
start_date = "2023-01-01"

# Generate the start of each month for the next 10 months
month_starts = pd.date_range(
    start=start_date, periods=10, freq="MS"
)  # 'MS' = Month Start

# Create a DataFrame with 10 empty columns (NaN values)
new_columns = pd.DataFrame(
    np.nan, index=final_port.index, columns=month_starts.strftime("%Y-%m-%d")
)

# Concatenate the existing DataFrame with the new columns
df_updated = pd.concat([final_port, new_columns], axis=1)
df_updated

,2023-01-01,2023-02-01,2023-03-01,2023-04-01,2023-05-01,2023-06-01,2023-07-01,2023-08-01,2023-09-01,2023-10-01
AAPL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MSFT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GOOG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import pandas as pd
import yfinance as yf


# Define the get_returns function
def get_returns(ticker, month):
    # Ensure ticker is a string
    ticker = str(ticker)

    # Use datetime to handle the month and year
    start_date = pd.to_datetime(month)
    end_date = start_date + pd.DateOffset(months=1)

    # Format dates as strings: 'YYYY-MM-DD' format
    start_date = start_date.strftime("%Y-%m-%d")
    end_date = end_date.strftime("%Y-%m-%d")

    # Download data for the ticker for the given month
    df_updated = yf.download(ticker, start=start_date, end=end_date, interval="1mo")

    # Check if the DataFrame is not empty and contains data
    if not df_updated.empty:
        # Calculate monthly return based on adjusted close prices
        df_updated["Returns"] = df_updated["Adj Close"].pct_change()

        # Return the monthly return (ignoring the first NaN value)
        return df_updated["Returns"].iloc[
            -1
        ]  # Only return the return for the last date in the month


# Input DataFrame with tickers as rows and months as columns
months = pd.date_range(start="2023-01-01", periods=12, freq="MS").strftime(
    "%Y-%m-%d"
)  # Monthly columns for 2023
tickers = ["AAPL", "MSFT", "GOOG"]  # Example tickers
df_updated = pd.DataFrame(index=tickers, columns=months)

# Function to populate returns for each stock and month
for ticker in df_updated.index:
    for month in df_updated.columns[1:]:  # Skip the first column (e.g., Jan)
        df_updated.at[ticker, month] = get_returns(ticker, month)

# Display the filled table
print(df_updated)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

     2023-01-01 2023-02-01 2023-03-01 2023-04-01 2023-05-01 2023-06-01  \
AAPL        NaN        NaN        NaN        NaN        NaN        NaN   
MSFT        NaN        NaN        NaN        NaN        NaN        NaN   
GOOG        NaN        NaN        NaN        NaN        NaN        NaN   

     2023-07-01 2023-08-01 2023-09-01 2023-10-01 2023-11-01 2023-12-01  
AAPL        NaN        NaN        NaN        NaN        NaN        NaN  
MSFT        NaN        NaN        NaN        NaN        NaN        NaN  
GOOG        NaN        NaN        NaN        NaN        NaN        NaN  


In [50]:
months = pd.date_range(start="2023-01-01", periods=12, freq="MS").strftime("%Y-%m-%d")
months

Index(['2023-01-01', '2023-02-01', '2023-03-01', '2023-04-01', '2023-05-01',
       '2023-06-01', '2023-07-01', '2023-08-01', '2023-09-01', '2023-10-01',
       '2023-11-01', '2023-12-01'],
      dtype='object')

In [51]:
print(get_returns("AAPL", "2023-06-01"))

[*********************100%***********************]  1 of 1 completed

nan


In [ ]:
### USE THISS

import yfinance as yf
import pandas as pd
from datetime import datetime


def get_returns(ticker, current_date):
    # Calculate the start of the year for the given date
    start_of_year = datetime(current_date.year, 1, 1)

    # Fetch data using yfinance
    data = yf.download(ticker, start=start_of_year, end=current_date, interval="1d")

    # Get the adjusted close price on the first day of the year
    start_price = data["Adj Close"].iloc[0]  # Price on the start of the year

    # Calculate the return as the percentage change from the start of the year
    current_price = data["Adj Close"].iloc[-1]  # Price on the current date

    pct_change = ((current_price / start_price) - 1) * 100  # Percentage change

    return pct_change


# Example of usage:
current_date = datetime.today()  # This will use today's date as the current month date
ticker = "AAPL"
print(
    f"Percentage change for {ticker} from the start of the year: {get_returns(ticker, current_date)}%"
)

[*********************100%***********************]  1 of 1 completed

Percentage change for AAPL from the start of the year: Ticker
AAPL    28.470523
dtype: float64%
